In [1]:
import pickle
import pandas as pd

In [2]:
with open("my_NYTimes_data20426.pkl", 'r') as picklefile: 
    my_old_data = pickle.load(picklefile)

In [177]:
key = my_old_data.keys()[0]

In [181]:
my_old_data.values()[0]['headline']

'Images That Capture the Leisure \xe2\x80\x94 and Loneliness \xe2\x80\x94 of College'

In [182]:
headlines = []
for i in my_old_data.values():
    headlines.append(i['headline'])

In [3]:
text = []
for i in my_old_data.values():
    text.append(i['text'])

In [75]:
my_old_data[key]['text']


20426

In [94]:
import numpy as np
mask = np.array(map(len, text)) > 0

##Using Dataframe

In [133]:
len(text[mask])

17848

In [97]:
text = np.array(text)

In [152]:
INDEX = np.random.choice(len(text[mask]), 5000)

In [105]:
from sklearn.feature_extraction.text import CountVectorizer
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=10000,
                                stop_words='english')
tf = tf_vectorizer.fit_transform(text[mask][INDEX])

In [153]:
import lda

model = lda.LDA(n_topics=30, n_iter=1500, random_state=1)
model.fit(tf)

<lda.lda.LDA instance at 0x104768ea8>

In [154]:
topic_word = model.topic_word_  # model.components_ also works
vocab = tf_vectorizer.get_feature_names()

In [155]:
print type(topic_word)
print topic_word.shape
print topic_word[0][:20]
print topic_word[0].sum()

<type 'numpy.ndarray'>
(30, 9610)
[  1.20176419e-06   1.20176419e-06   1.20176419e-06   1.20176419e-06
   1.20176419e-06   1.20176419e-06   1.20176419e-06   1.20176419e-06
   1.20176419e-06   1.20176419e-06   1.20176419e-06   1.20176419e-06
   1.20176419e-06   2.41554602e-04   1.20176419e-06   1.20176419e-06
   1.20176419e-06   1.20176419e-06   1.20176419e-06   1.20176419e-06]
1.0


In [156]:
n_top_words = 10
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: said new including year months national month media did
Topic 1: mr obama president republican political party campaign power white
Topic 2: day 10 time year years second just 30 little
Topic 3: dr drug patients care heart hospital medical drugs percent
Topic 4: police city officers new york public officer stop department
Topic 5: climate dr scientists change clouds study tree researchers warming
Topic 6: horse meat grams stir oil chicken add fat water
Topic 7: trump history clinton hillary trumps political play deeply donald
Topic 8: children school young students life age families child kids
Topic 9: people years like make long does says way little
Topic 10: groups islamic state muslim syria supporters isis group social
Topic 11: league france players games spanish premier ball sports years
Topic 12: company billion business deal million stock yahoo executive financial
Topic 13: mr times news page editor new time report pages
Topic 14: team game season play finals teams play

In [175]:
# document topic distributions
doc_topic = model.doc_topic_ 

In [176]:
for i in range(15):
    print("{} (top topic: {})".format(text[mask][i], doc_topic[i].argmax()))

Sprawled on grass, floating down a river, or gazing blankly into the distance  all the subjects captured, unawares, in the photographer Paddy Summerfields new book The Oxford Pictures share a certain listlessness. In 1968, having been thrown out of Guildford School of Art (the staff werent particularly sympathetic towards my vision), Summerfield returned to his hometown of Oxford, England. He spent the summers of the next 10 years wandering around the grounds of the elite Oxford University, where he photographed students at leisure.What he sensed at the university, he says, was an atmosphere that mirrored how he felt about his own life. I was young, he says, Its a young persons vision, noticing girls and noticing other peoples relationships  but I was always outside everything. He recognized a similar nervousness in the subjects of his photos, who hovered between the social rituals of university and an existential uncertainty. At the time, Summerfield was heavily influenced by John Len

In [168]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic {0}:".format(topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [ ]:
print_top_words()

##SKLEARN

In [158]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

ImportError: cannot import name LatentDirichletAllocation

In [169]:
from sklearn import feature_extraction
sklearn.__version__

'0.16.1'

In [171]:
!pip install scikit-learn==0.17.1

In [172]:
!echo $PATH

/Users/matthewmitchell/anaconda/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/usr/texbin


In [166]:
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /Users/matthewmitchell/anaconda/lib/python2.7/site-packages


In [163]:
!conda install scikit-learn

Fetching package metadata .......
Solving package specifications: ..........

# All requested packages already installed.
# packages in environment at /Users/matthewmitchell/anaconda:
#
scikit-learn              0.17.1              np111py27_2  


##Calculating Article Reading Grade Levels Using Flescher Kincaid

In [13]:
import textstat.textstat

In [3]:
nyt_df = pd.DataFrame.from_dict(my_old_data, orient='index')

In [5]:
number_failures = len(nyt_df[nyt_df.text==''])
number_successes = len(nyt_df[nyt_df.text!=''])
success_percent = float(number_successes)/(number_failures+number_successes)
print 'Percent of Articles successfully scraped:', success_percent
print 'Number of Articles successfully scraped:', number_successes

Percent of Articles successfully scraped: 0.873788309018
Number of Articles successfully scraped: 17848


In [ ]:
# re.sub("[.,?!](?=[A-Z])", ". ", nyt_df.ix[1].text)

In [6]:
has_text = nyt_df[nyt_df.text!='']

In [7]:
len(has_text)

17848

In [16]:
grade_levels = [textstat.textstat.textstat.flesch_kincaid_grade(has_text.ix[index].text) for index in range(len(has_text))]

In [92]:
len(grade_levels)

4605

In [18]:
has_text['grade_levels'] = grade_levels

/Users/matthewmitchell/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [21]:
has_text[has_text.grade_levels==6.0]

,headline,text,abstract,locations,word_count,id,snippet,source,subjects,url,desk,date,type,section,grade_levels
http://6thfloor.blogs.nytimes.com/2013/08/09/the-day-jerry-garcia-died/,The Day Jerry Garcia Died,We were driving down Knoxville Avenue in Peori...,It’s been 18 years since Jerry Garcia died. Ha...,[],544,5205548579881007d590276e,It’s been 18 years since Jerry Garcia died. Ha...,The New York Times,[],http://6thfloor.blogs.nytimes.com/2013/08/09/t...,None,2013-08-09,Blog,Magazine,6
http://6thfloor.blogs.nytimes.com/2014/05/28/want-to-write-a-poem-with-patricia-lockwood/,Want to Write Verse With Patricia Lockwood?,"In his profile of Patricia Lockwood, Jesse Lic...",Join us as in a collective poetry-writing sess...,[],615,53866ca938f0d83ab04903ea,Join us as in a collective poetry-writing sess...,The New York Times,[Poetry and Poets],http://6thfloor.blogs.nytimes.com/2014/05/28/w...,Magazine,2014-05-28,Blog,Magazine,6
http://artsbeat.blogs.nytimes.com/2013/10/22/kerry-james-evans-from-combat-engineer-to-poet/,Kerry James Evans: From Combat Engineer to Poet,"Kerry James Evans, whos just 30, has hard-knoc...","Mr. Evans talks about “Bangalore,” his first c...",[],1001,5266ffa638f0d86e0b755905,"Mr. Evans talks about “Bangalore,” his first c...",The New York Times,[Poetry and Poets],http://artsbeat.blogs.nytimes.com/2013/10/22/k...,None,2013-10-22,Blog,Arts,6
http://artsbeat.blogs.nytimes.com/2014/06/19/memorial-concert-scheduled-for-bitter-end-owner/,Memorial Concert Scheduled for Bitter End Owner,"Friends of Paul Colby, the owner of the Bitter...","Friends of Paul Colby, the owner of the Bitter...",[],146,53a303e77988101bf370bbb7,"Friends of Paul Colby, the owner of the Bitter...",The New York Times,[Music],http://artsbeat.blogs.nytimes.com/2014/06/19/m...,Culture,2014-06-19,Blog,Arts,6
http://artsbeat.blogs.nytimes.com/2015/02/27/house-of-cards-season-3-episode-4-recap-solicitor-general-blues/,&#8216;House of Cards&#8217; Season 3 Episode ...,"Season 3, Episode 4: Chapter 30Do you know who...",Do you know who the solicitor general is? Do y...,[],1081,54f0f3df79881073aef9ea5a,Do you know who the solicitor general is? Do y...,The New York Times,[Television],http://artsbeat.blogs.nytimes.com/2015/02/27/h...,Culture,2015-02-27,Blog,Arts,6
http://artsbeat.blogs.nytimes.com/2015/12/14/adele-25-billboard-three-week-record/,Adele&#8217;s Third Week at No. 1 Is Accompani...,"Another week, another eye-popping sales statis...",Coldplay’s “A Head Full of Dreams” debuts at N...,[],266,566f291d79881010750bc1ef,Coldplay’s “A Head Full of Dreams” debuts at N...,The New York Times,[],http://artsbeat.blogs.nytimes.com/2015/12/14/a...,Culture,2015-12-14,Blog,Arts,6
http://bits.blogs.nytimes.com/2012/10/10/timescast-mediatech-electric-motorcycles/,TimesCast Media+Tech: Electric Motorcycles,The strategy behind the networks fall dramas. ...,The strategy behind the networks' fall dramas....,[],25,507c12ae95e998af7117ff8d,The strategy behind the networks' fall dramas....,The New York Times,"[Mobile Applications, Motorcycles, Motor Bikes...",http://bits.blogs.nytimes.com/2012/10/10/times...,None,2012-10-10,Blog,Technology,6
http://boss.blogs.nytimes.com/2012/01/31/doing-the-math-on-disappointed-customers/,Doing the Math on Disappointed Customers,"In the last 30 years, I have made almost every...",First there are the referrals that you lose. T...,[],942,4fd3a2298eb7c8105d8e9c86,First there are the referrals that you lose. T...,The New York Times,"[Small Business, Guarantees and Warranties, Cu...",http://boss.blogs.nytimes.com/2012/01/31/doing...,None,2012-01-31,Blog,Business Day,6
http://boss.blogs.nytimes.com/2012/07/02/learning-from-titos-vodka/,Learning From Tito's Vodka,"In my last post, I talked about how Tito Bever...",Tito Beveridge proved you do not necessarily n...,[Austin (Tex)],627,50008d391c22dfde670af5a8,Tito Beveridge proved you do not necessarily n...,The New York Times,"[Advertising and Marketing, Small Business, Vo...",http://boss

In [20]:
has_text.sort_index(by='grade_levels', axis=0).tail(20)

,headline,text,abstract,locations,word_count,id,snippet,source,subjects,url,desk,date,type,section,grade_levels
http://thelede.blogs.nytimes.com/2012/05/03/qaeda-leadership-considered-rebranding/,Qaeda Leadership Considered Rebranding,"As my colleague Peter Baker reports, one of th...",To make Al Qaeda's connection to Islam more ex...,[],779,4fd3a4aa8eb7c8105d8ef7ee,To make Al Qaeda's connection to Islam more ex...,The New York Times,[],http://thelede.blogs.nytimes.com/2012/05/03/qa...,None,2012-05-03,Blog,None,20.5
http://www.nytimes.com/2016/05/29/arts/design/rembrandts-first-masterwork-on-display-at-the-morgan-library.html,Rembrandt’s First Masterwork on Display at the...,The Dutch diplomat and poet Constantijn Huygen...,NaN,[],135,5747606479881005c7f206ce,“Judas Returning the Thirty Pieces of Silver” ...,The New York Times,[Art],http://www.nytimes.com/2016/05/29/arts/design/...,Arts&Leisure,2016-05-29,News,Arts,20.5
http://cityroom.blogs.nytimes.com/2015/01/07/25-injured-in-brooklyn-bus-crash-2/,Numerous Injuries in Brooklyn Bus Crash,Four men were critically injured and more than...,Six people were seriously injured and 19 other...,"[Accidents and Safety, Brooklyn (NYC), Buses]",41,54ade9e07988105cf609525f,Six people were seriously injured and 19 other...,The New York Times,[],http://cityroom.blogs.nytimes.com/2015/01/07/2...,Metro,2015-01-07,Blog,N.Y. / Region,20.5
http://www.nytimes.com/2015/09/09/sports/olympian-writes-of-positive-doping-test.html,Olympian Writes of Positive Doping Test,"Clara Hughes, a former cyclist and speedskater...","Former Olympian, Canadian Clara Hughes, reveal...",[],48,55efab1a7988104d070a1966,"Clara Hughes, a former cyclist and speedskater...",The New York Times,"[Books and Literature, Doping (Sports), Olympi...",http://www.nytimes.com/2015/09/09/sports/olymp...,Sports,2015-09-09,Brief,Sports,20.8
http://www.nytimes.com/2015/10/20/sports/cricket/umpire-pulled-after-threat.html,Umpire Pulled After Threat,The International Cricket Council withdrew the...,NaN,[],42,5625ba937988101bb67566be,The International Cricket Council withdrew the...,The New York Times,"[Officiating (Sports), Cricket (Game)]",http://www.nytimes.com/2015/10/20/sports/crick...,Sports,2015-10-20,Brief,Sports,20.8
http://learning.blogs.nytimes.com/2014/06/04/year-end-round-up-2013-14-math-science-health-and-technology/,"Year-End Roundup, 2013-14 | Math, Science, Hea...","This week, were gathering all the lesson plans...","All the Science, Math, Health and Technology l...",[],575,538f4fc538f0d86de266be99,"All the Science, Math, Health and Technology l...",The New York Times,[],http://learning.blogs.nytimes.com/2014/06/04/y...,None,2014-06-04,Blog,U.S.,20.9
http://www.nytimes.com/2012/07/12/world/europe/britain-olympics-duty-for-3500-more-troops.html,"Britain: Olympics Duty for 3,500 More Troops",Barely two weeks before the opening of the Oly...,"British government drafts an additional 3,500 ...","[Europe, Great Britain]",97,515b4ab6fdc59278db007858,A major shortfall in the number of private sec...,The New York Times,"[Olympic Games, Terrorism, Olympic Games (2012)]",http://www.nytimes.com/2012/07/12/world/europe...,Foreign,2012-07-12,Brief,World,22.0
http://www.nytimes.com/2015/10/21/t-magazine/art-tokyo-ugo-rondinone-john-giorno.html,A Love Letter to John Giorno,The artist Ugo Rondinone adores the poet John ...,NaN,[Paris (France)],156,5627af7d38f0d85d3d8b392b,"Part retrospective, part shrine to his partner...",The New York Times,"[Poetry and Poets, Art]",http://www.nytimes.com/2015/10/21/t-magazine/a...,TStyle,2015-10-21,News,T Magazine,22.0
http://tmagazine.blogs.nytimes.com/2015/02/06/fady-joudah-tatiana-trouve-picture-poem/,On Atonement,ALMOST YOUR LIFEThen another fled resemblancea...,"In a mirror’s reflection, the artist Tatiana T...",[],141,54d504d638f0d85e2b8aee16,"In a mirror’s reflection, the artist Tatiana T...",The New York Times,[Poetry and Poets],http://tmagazine.blogs.nytimes.com/2015/02/06/...,TStyle,2015-02-06,

In [100]:
has_text[has_text.url=='http://www.nytimes.com/2012/07/30/opinion/keller-the-entitled-generation.html']

,headline,text,abstract,locations,word_count,id,snippet,source,subjects,url,desk,date,type,section,grade_levels
http://www.nytimes.com/2012/07/30/opinion/keller-the-entitled-generation.html,The Entitled Generation,"IF you were born before 1946 or after 1964, yo...",Bill Keller Op-Ed column reflects on the amoun...,[],1360,515b4648fdc59278db007304,"Boomers have been under attack, but there are ...",The New York Times,"[Baby Boomers, Federal Budget (US), Federal Ai...",http://www.nytimes.com/2012/07/30/opinion/kell...,Editorial,2012-07-30,Op-Ed,Opinion,10.3


##BNC Corpus

In [4]:
word_freq_df = pd.read_csv('word_freq.txt', delim_whitespace=True)
word_freq_df.columns = ['Frequency','Word','Parts_of_Speach','Num_doc_occurences']
word_freq_df = (word_freq_df[~word_freq_df.Word.isnull()])

In [5]:
print len(word_freq_df)
word_freq_df.head()

938964


,Frequency,Word,Parts_of_Speach,Num_doc_occurences
0,6187267,the,at0,4120
1,2941444,of,prf,4108
2,2682863,and,cjc,4120
3,2126369,a,at0,4113
4,1812609,in,prp,4109


##Stemming the British Word Counts data...

In [6]:
from nltk.stem import porter
from nltk.stem import LancasterStemmer
import re

In [7]:
stemmer = porter.PorterStemmer()
stemmerL = LancasterStemmer()

In [8]:
Stemmed = [stemmer.stem(word) for word in word_freq_df.Word]
word_freq_df['Word_stem'] = Stemmed

In [9]:
symbols = re.compile("[^a-zA-Z]")
word_freq_df = word_freq_df[~word_freq_df.Word.str.contains(symbols)]

In [10]:
word_freq_df = word_freq_df.reset_index()

In [11]:
word_freq_df.tail()

,index,Frequency,Word,Parts_of_Speach,Num_doc_occurences,Word_stem
561254,862326,1,aaaaaaaaaaaaaaahhhhhhhhhhhhhhhhhhh,nn1,1,aaaaaaaaaaaaaaahhhhhhhhhhhhhhhhhhh
561255,862328,1,aaaa,nn0,1,aaaa
561256,863519,1,a,vhb,1,a
561257,863520,1,a,pnp,1,a
561258,863521,1,a,aj0,1,a


In [68]:
word_freq_df[word_freq_df.Word=='table']

,index,Frequency,Word,Parts_of_Speach,Num_doc_occurences,Word_stem
433,456,20142,table,nn1,2365,tabl
66138,72425,35,table,nn1-vvb,27,tabl
94137,105675,18,table,np0,12,tabl
194885,237902,4,table,vvi,3,tabl
360818,506879,1,table,vvb,1,tabl


In [12]:
grouped = word_freq_df.groupby('Word_stem', as_index=False).sum()

In [13]:
grouped.sort_index(by='Frequency').tail()

,Word_stem,index,Frequency,Num_doc_occurences
0,a,4951463,2150886,7102
221178,to,5026902,2560384,13326
7545,and,7071737,2683114,4231
156128,of,2629802,2941868,4485
218414,the,1503467,6187941,4411


In [14]:
grouped[grouped['Word_stem']=='table']

,Word_stem,index,Frequency,Num_doc_occurences
214201,table,506826,1,1


In [15]:
stem_freq_dict = grouped.set_index('Word_stem')['Frequency'].to_dict()

In [16]:
word_freq_df.head()

,index,Frequency,Word,Parts_of_Speach,Num_doc_occurences,Word_stem
0,0,6187267,the,at0,4120,the
1,1,2941444,of,prf,4108,of
2,2,2682863,and,cjc,4120,and
3,3,2126369,a,at0,4113,a
4,4,1812609,in,prp,4109,in


In [17]:
len(word_freq_df)

561259

##Stanford Dictionary of English Words; Will Remove Words from BNC not in Dictionary

In [18]:
stanford_word_list = pd.read_csv('dictionary_stanford.txt', header=None)
stanford_word_list.columns=['Word']

In [19]:
any(stanford_word_list.Word=='aa')

True

In [20]:
print len(word_freq_df)
print len(stanford_word_list)

561259
127142


In [21]:
# words_to_drop=[]
# for word in word_freq_df.Word:
#     if any(stanford_word_list['Word']==word):
#         continue
#     else:
#         words_to_drop.append(word)

In [22]:
stemmed_dict_word_list = [stemmer.stem(word) for word in stanford_word_list.Word]


In [23]:
stanford_word_list['Word_stem'] = stemmed_dict_word_list

In [24]:
stanford_word_list.head()

,Word,Word_stem
0,aa,aa
1,aah,aah
2,aahed,aah
3,aahing,aah
4,aahs,aah


In [25]:
stemmed_dict_list = list(stanford_word_list.Word_stem)

In [26]:
final_word_df = grouped[grouped.Word_stem.isin(stemmed_dict_list)]

In [27]:
final_word_df = final_word_df.reset_index()

In [28]:
final_word_df.tail()

,level_0,Word_stem,index,Frequency,Num_doc_occurences
37552,248126,zygot,1247371,23,13
37553,248127,zygoten,452503,1,1
37554,248134,zymogen,589982,12,4
37555,248135,zymot,452495,1,1
37556,248137,zymurgi,904985,2,2


In [29]:
final_word_df.sort_index(by='Frequency').tail()

,level_0,Word_stem,index,Frequency,Num_doc_occurences
16079,104791,in,1750955,1883465,11607
33471,221178,to,5026902,2560384,13326
1051,7545,and,7071737,2683114,4231
22016,156128,of,2629802,2941868,4485
33017,218414,the,1503467,6187941,4411


In [30]:
final_word_df[final_word_df['Word_stem']=='the']

,level_0,Word_stem,index,Frequency,Num_doc_occurences
33017,218414,the,1503467,6187941,4411


In [88]:
stem_freq_dict = final_word_df.set_index('Word_stem')['Frequency'].to_dict()

In [ ]:
# to_drop = []
# i_list = range(len(word_freq_df))
# # j_list = range(len(words_df))
# for i in i_list:
#     for j in i_list[i+1:]:
#         if word_freq_df.ix[i].Word_stem == word_freq_df.ix[j].Word_stem:
#             word_freq_df.ix[i].Frequency += word_freq_df.ix[j].Frequency
#             to_drop.append(j)

##Create a dictionary of our word frequencies

In [354]:
# df.set_index('id')['value'].to_dict()
freq_dict = grouped_words['Frequency'].to_dict()

##For a given article, what are the most difficult words?

In [314]:
# Adding spaces after periods
# test_article = re.sub("[.,?!](?=[A-Z])", ". ", test_article)

In [43]:
text[5]

'In a small Brooklyn apartment, Jacob Winterhalt, a former Air Force airman, looked on as his 3-year-old daughter, Raya, and her best friend, Doris, chased each other. The girls wore sneakers. Doriss were equipped with red lights that flashed with the urgency of a distress beacon.No shoes in the house, you silly goose, Mr. Winterhalt said. Unless those are special shoes.Ive got to go to work, Raya said.Ive got to go to work, too, Doris said.O.K., said Mr. Winterhalt, 35. Have a good day.With that, he sat down at the kitchen table. The girls went back to running around the tight space as Sampa, a 7-year-old English bulldog, barked at no one in particular. Nearby, Doriss father, Joachim Carels, 39, who used to work in the fashion industry, lounged on the couch.At that moment, it was not lost on me that taking care of the two little ones had been the work of both men for the last three years.As their children grew close, these two men from very different backgrounds developed a friendship

In [92]:
text[18]

'IF you were born before 1946 or after 1964, you are free to go. Kindly close the door on your way out. I need a private moment with my fellow baby boomers.So. I imagine youre all feeling a little unappreciated these days. We seem to have entered one of our periodic seasons of boomer-bashing. In rapid Op-Ed succession, we children of the postwar demographic bulge have been blamed for turning religion into an indulgent free-for-all, for giving elites a bad name and for making greed respectable, or at least acceptable. Thats just this month, and just on this page. And its not only conservatives beating us with the Woodstock whip. Kurt Andersen, a confessed liberal and one of our more prolific cultural omnivores, started the latest thumping July 4 with an argument that amoral self-gratification is just the flip side of social liberation: Thanks to the 60s, we are all shamelessly selfish.The notion that our generation has been spoiled rotten is not a terribly new thought. A dozen years ago

In [120]:
words=[]
article_word_list = text[18].replace("(","").replace(")","").replace("-"," ").split()
cleaned_and_stemmed = [(stemmer.stem(word), word) for word in article_word_list]
for stem, word in cleaned_and_stemmed:
    word = word.lower()
    word = word.strip(',.')
    stem= stem.lower()
    stem = stem.strip(',.')
    try:
        frequency = stem_freq_dict[stem]
        words.append((word, frequency))
    except:
        continue

for word, frequency in sorted(set(words), key=lambda x: x[1])[:6]:
    print word, 'BNC Frequency:', frequency

    

catchall BNC Frequency: 2
aggrandizing BNC Frequency: 3
deathwatch BNC Frequency: 6
succession BNC Frequency: 9
lodestar BNC Frequency: 10
psychodrama BNC Frequency: 11


In [109]:
stemmer.stem('moment')

u'moment'

In [108]:
text[256]

'ARCHITECTURES ODD COUPLEOdd coupling did not begin with Felix and Oscar. Think of all the rivals shackled together by history: Bernini and Borromini, Hemingway and Fitzgerald, Robert Moses and Jane Jacobs. The pairing of frenemies is a time-honored storytelling device used by biographers to help us understand their subjects more fully. Its as if we can truly know them only by their differences, their personal grievances and philosophical feuds. The historian Hugh Howard now applies this approach, in Architectures Odd Couple, to two of architectures most outsize personalities, Frank Lloyd Wright and Philip Johnson.Even as odd couples go, Wright and Johnson seem an unlikely choice for a dual biography. The two men interacted with so many cultural giants that Howard could have yoked each to any number of others  Wright and Louis Sullivan, Johnson and Mies van der Rohe. Then there is the matter of the achievement gap. Wright is one of the greatest architects of all time. While Johnson is 

In [118]:
words=[]
article_word_list = text[256].replace("(","").replace(")","").replace("-"," ").split()
cleaned_and_stemmed = [(stemmer.stem(word), word) for word in article_word_list]
for stem, word in cleaned_and_stemmed:
    word = word.lower()
    word = word.strip(',.')
    stem= stem.lower()
    stem = stem.strip(',.')
    try:
        frequency = stem_freq_dict[stem]
        words.append((word, frequency))
    except:
        continue

for word, frequency in sorted(set(words), key=lambda x: x[1])[0:6]:
    print word, 'BNC Frequency:', frequency

    

sanitarium BNC Frequency: 1
porkpie BNC Frequency: 2
oud BNC Frequency: 5
mies BNC Frequency: 24
crisscrossing BNC Frequency: 24
catalog BNC Frequency: 25


In [119]:
has_text[has_text.url=='http://www.nytimes.com/2016/07/03/books/review/architectures-odd-couple-philip-johnson-and-frank-lloyd-wright.html']

,headline,text,abstract,locations,word_count,id,snippet,source,subjects,url,desk,date,type,section,grade_levels
http://www.nytimes.com/2016/07/03/books/review/architectures-odd-couple-philip-johnson-and-frank-lloyd-wright.html,"In ‘Architecture’s Odd Couple,’ It’s Philip Jo...",ARCHITECTURES ODD COUPLEOdd coupling did not b...,NaN,[],1312,57768f607988101bbb4b9a97,"Two outsize personalities, Frank Lloyd Wright ...",The New York Times,"[Books and Literature, Architecture]",http://www.nytimes.com/2016/07/03/books/review...,BookReview,2016-07-03,Review,Books,9.9


In [116]:
text[1258]

'\n\nWhat does it mean to multiply? We often think of multiplication as creating a kind of grid: four groups of three oranges gives us twelve oranges. This is the way multiplication is often taught  but are there other ways to think about it? This question came up recently in conversation with Kalid Azad of BetterExplained.com. Heres Mr. Azads response:I see multiplication as a way to combine the properties of various numbers. The number 2 has two-ness and the number 3 has three-ness.Multiply them together, and the result (6) will have both two-ness and three-ness, i.e. its divisible by either. We can multiply by 2 again and get 12, which has double two-ness and a single three-ness.Heres how it helps me: Remember the rules like odd x odd = odd and odd x even = even? Why is that?Well, an odd number has no two-ness. Multiplying by another odd doesnt add any two-ness, so it stays odd. But odd x even? Well, we just mixed some two-ness into the batter. The result *must* be even because we p

In [117]:
words=[]
article_word_list = text[1258].replace("(","").replace(")","").replace("-"," ").split()
cleaned_and_stemmed = [(stemmer.stem(word), word) for word in article_word_list]
for stem, word in cleaned_and_stemmed:
    word = word.lower()
    word = word.strip(',.')
    stem= stem.lower()
    stem = stem.strip(',.')
    try:
        frequency = stem_freq_dict[stem]
        words.append((word, frequency))
    except:
        continue

for word, frequency in sorted(set(words), key=lambda x: x[1])[0:6]:
    print word, 'BNC Frequency:', frequency

favorite BNC Frequency: 28
primo BNC Frequency: 69
unsung BNC Frequency: 100
ness BNC Frequency: 213
unnamed BNC Frequency: 239
color BNC Frequency: 335


In [115]:
set(article_word_list)

{'#1',
 '#2',
 '#3,',
 '#4,',
 '*must*',
 '+',
 '+16',
 '0',
 '0,',
 '1',
 '1,',
 '1,1,',
 '1.',
 '10',
 '10.',
 '100',
 '100,',
 '100.',
 '101',
 '101,',
 '11',
 '11,',
 '12',
 '12,',
 '14',
 '16',
 '16.',
 '19',
 '1:',
 '2',
 '2,',
 '2,3;',
 '2,4,',
 '2.',
 '2.Golden',
 '24',
 '24,',
 '24.',
 '24.Ravi',
 '28,',
 '2:',
 '3',
 '3,4;',
 '3.',
 '32',
 '34%',
 '35%',
 '4',
 '4,',
 '4,5,',
 '4.',
 '5',
 '5.',
 '56.',
 '6',
 '61.',
 '64,',
 '65',
 '65,',
 '7',
 '8',
 '9',
 '9,10;',
 '91',
 '96',
 '99',
 '99,',
 '=',
 'Allen',
 'And',
 'As',
 'At',
 'Azad',
 'Azads',
 'BetterExplained.com.',
 'But',
 'Can',
 'Cook:Math',
 'Crunching',
 'Dan',
 'Dice',
 'Do',
 'Dr.',
 'Dragon',
 'Dragon,',
 'Each',
 'Enhancer',
 'Finkel',
 'Finkel,',
 'First',
 'For',
 'Fourth',
 'From',
 'Gary',
 'Golden',
 'He',
 'Heres',
 'Hers',
 'Hewitts',
 'How',
 'I',
 'If',
 'Ill',
 'In',
 'Its',
 'Kalid',
 'Katherine',
 'Love:',
 'Loves',
 'Martin.',
 'Math',
 'Move',
 'Mr.',
 'Multiplying',
 'No',
 'On',
 'Or',
 'Pl

In [39]:
test_article = 'Spanish-speaking audiences didnt turn out for Los Monlogos de la Vagina, forcing the quick closure of Eve Enslers play after 15 previews and 33 regular performances. Producers hoped that importing a translated Vagina Monologues that has been playing 14 years in Mexico City, and bringing in Spanish-speaking actors with followings from stage and television, would be a recipe for renewed New York success. (The play initially ran for 1,300 performances.)  They described the production as the first open commercial run of a Spanish-language play Off Broadway. But that came to a close with Mondays announcement that the Sunday night performance had been the shows last. Even with positive notices and enthusiastic audiences, the show is not attracting the numbers needed to make this economically viable,  the producers said in a statement. We hope to announce plans to bring the production to other U. S. cities.'

In [40]:
words=[]
for word in test_article.replace("(","").replace(")","").replace("-"," ").replace("nt","").split():
    word = word.lower()
    word = word.strip(',.')
    try:
        frequency = freq_dict[word]
        words.append((word, frequency))
    except:
        continue

print sorted(set(words), key=lambda x: x[1])
    

[('recipe', 1), ('that', 1), ('this', 1), ('out', 1), ('a', 1), ('the', 1), ('success', 1), ('is', 1), ('spanish', 1), ('notices', 1), ('of', 1), ('not', 1), ('monologues', 1), ('other', 1), ('off', 1), ('came', 1), ('importing', 1), ('show', 1), ('from', 1), ('and', 1), ('television', 1), ('did', 1), ('but', 1), ('make', 1), ('first', 1), ('for', 1), ('open', 1), ('close', 1), ('quick', 1), ('as', 1), ('positive', 1), ('regular', 1), ('closure', 1), ('to', 1), ('be', 1), ('s', 1), ('attracting', 1), ('translated', 2), ('plans', 2), ('turn', 2), ('economically', 2), ('vagina', 2), ('la', 2), ('even', 3), ('renewed', 3), ('last', 3), ('ran', 3), ('york', 3), ('u', 4), ('run', 4), ('hope', 4), ('previews', 4), ('performance', 5), ('broadway', 5), ('play', 5), ('in', 5), ('forcing', 6), ('needed', 6), ('had', 6), ('has', 6), ('language', 7), ('new', 7), ('after', 8), ('night', 8), ('numbers', 8), ('described', 9), ('would', 10), ('production', 10), ('eve', 12), ('mondays', 12), ('stage', 

In [41]:
text[2]

NameError: name 'text' is not defined

In [197]:
words=[]
for word in text[2].split():
    word = word.lower()
    word = word.strip(',.')
    try:
        frequency = freq_dict[word]
        if any(dictionary['Word']==word):
            words.append((word, frequency))
    except:
        continue

print sorted(set(words), key=lambda x: x[1])[:10]
    

[]


In [44]:
def hardest_words(article):
    words=[]
    for word in article.replace("(","").replace(")","").replace("-"," ").replace("nt","").split():
        word = word.lower()
        word = word.strip(',.')
        try:
            frequency = freq_dict[word]
            if any(dictionary['Word']==word):
                words.append((word, frequency))
        except:
            continue

    print sorted(set(words), key=lambda x: x[1])[:10]

In [45]:
hardest_words(text[5])

[]


In [194]:
len(dictionary)

127142

##Dictionary of 10,000 most common English Words

In [415]:
common_dictionary = pd.read_csv('google-10000-english.txt', header=None)
dictionary.columns=['Word']

In [416]:
common_dictionary.head()

,0
0,the
1,of
2,and
3,to
4,a


##MongoDB

In [ ]:
has_text.to_json